In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

In [12]:
def fit_error(y, f):
    return np.sum((y-f)**2)

In [13]:
def fit_func_1(Z, A, sigma):
    
    P = (A/np.sqrt(2*np.pi*sigma**2))*np.exp(-Z**2/(2*sigma**2))
    
    return P

In [14]:
def fit_func_2(Z, B, alpha):
    
    P = B*np.exp(-alpha*Z)
    
    return P

In [15]:
def fit_PDF(centers, heights):
    
    # Information about the peak in the numerical PDF
    peak_height = np.max(heights)
    peak_ind = np.where(heights == peak_height)[0][0]
    
    # mu is where the numerical PDF peaks
    mu = centers[peak_ind]
 
    # Recentering so that peak is at z = 0
    del_centers = centers - mu
    gauss_centers = del_centers[0:peak_ind+1]
    gauss_heights = heights[0:peak_ind+1]
                
    fit = np.zeros(len(centers))
    fit_err = np.inf
    fit_params = np.zeros(5)
    
    for i in range(0, len(fit_params)):
        fit_params[i] = float('nan')
    
    try:
        
        # Estimating sigma using FWHM
        sigma = 0

        for i in range(0, len(gauss_centers)):
            if(heights[i] >= peak_height/2):
                sigma = (-gauss_centers[i])/np.sqrt(2*np.log(2))
                break

        # Estimating A accordingly, by using the peak height
        A = np.sqrt(2*np.pi*sigma**2)*peak_height

        # First fit a Gaussian

        guess_gauss_params = np.array([A, sigma])
        
        gauss_bounds = ([0, 0], [np.max(heights), np.max(np.abs(del_centers))])

        fit_gauss_params, fit_gauss_covar = optimize.curve_fit(fit_func_1, gauss_centers, gauss_heights,
                                                               p0 = guess_gauss_params, 
                                                               bounds = gauss_bounds, method = 'trf')

                
        fit_params[0] = fit_gauss_params[0]
        fit_params[1] = mu
        fit_params[2] = fit_gauss_params[1]

        fit = fit_func_1(del_centers, *fit_gauss_params)
        fit_err = fit_error(heights, fit)

        # See if an exponential decay tail exists and is a better fit

        for i in range(peak_ind, len(centers)):

            try:

                v_T = centers[i]

                exp_centers = del_centers[i:] - del_centers[i]

                exp_heights = heights[i:]
                init_height = exp_heights[0]

                # Estimating alpha using half-life decay
                alpha = 0

                for j in range(0, len(exp_centers)):
                    if(exp_heights[j] <= init_height/2):
                        alpha = np.log(2)/(exp_centers[j])
                        break

                B = (fit_params[0]*np.exp(alpha*v_T))/np.sqrt(2*np.pi*fit_params[2]**2)*np.exp(-(v_T-mu)**2/(2*fit_params[2]**2))

                guess_exp_params = np.array([B, alpha])
                
                exp_bounds = ([0, 0], [np.max(heights), np.inf])

                fit_exp_params, fit_exp_covar = optimize.curve_fit(fit_func_2, exp_centers, exp_heights,
                                                                   p0 = guess_exp_params, 
                                                                   bounds = exp_bounds, method = 'trf')

                gauss_fit = fit_func_1(del_centers[:i], *fit_gauss_params)
                exp_fit = fit_func_2(exp_centers, *fit_exp_params)
                curr_fit = np.concatenate([gauss_fit, exp_fit])
                curr_err = fit_error(heights, curr_fit)

                if(curr_err < fit_err):
                    fit = curr_fit
                    fit_err = curr_err
                    fit_params[3] = fit_exp_params[1]
                    fit_params[4] = v_T

            except:
                continue
                
    except:
        
        pass
    
    return fit, fit_params

In [16]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m12i_res7100/


In [17]:
# Get all rendered indices

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')
rendered_indices = rendered_df['snap'].to_list()
redshifts = rendered_df['redshift'].to_list()
num_snaps = len(rendered_indices)

fitted_df = rendered_df

# Get rendered metals

metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

# Create a list of paths for all metals
spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [18]:
# Get statistics for all snapshots
As = {}
mus = {}
sigmas = {}
alphas = {}
z_Ts = {}

for m in metals:
    As[m] = []
    mus[m] = []
    sigmas[m] = []
    alphas[m] = []
    z_Ts[m] = []
    
for i in range(0, num_snaps):
    
    snap_index = rendered_indices[i]
    redshift = redshifts[i]

    print('Processing snapshot {} ... \n'.format(str(snap_index)))

    snap_info = {}

    for m in metals:
        snap_info[m] = {}

    
    # Get numerical data for all metals
    for m in metals:

        print('Processing {} ...'.format(m.title()))

        # Load the numerical PDF and normalize

        num_df = pd.read_csv(spath_metals[m] +'data/num/' + str(snap_index) + '-num_' + m + '_data.csv')
        centers = np.array(num_df['abundance'].to_list())
        heights = np.array(num_df['num_val'].to_list())

        mass_norm = np.max(heights)
        heights /= mass_norm

        # Compute the fitted PDF

        fit, fit_params = fit_PDF(centers, heights)

        print(fit_params)

        # Rescaling range to achieve desired normalization

        heights *= mass_norm
        fit *= mass_norm
        fit_params[0] *= mass_norm

        # Appending fitted data
        snap_info[m]['centers'] = centers
        snap_info[m]['heights'] = heights
        snap_info[m]['fit'] = fit
        snap_info[m]['fit_params'] = fit_params


    for m in metals:

        # Saving the fitted data

        datafile =  str(snap_index) + '-fit_' + m + '_data' + '.csv'

        centers = snap_info[m]['centers']
        heights = snap_info[m]['heights']
        fit = snap_info[m]['fit']
        fit_params = snap_info[m]['fit_params']

        As[m].append(fit_params[0])
        mus[m].append(fit_params[1])
        sigmas[m].append(fit_params[2])
        z_Ts[m].append(fit_params[3])
        alphas[m].append(fit_params[4])

        fit_dict = {'abundance': centers, 'fit_val': fit}
        fit_df = pd.DataFrame(fit_dict)
        fit_df.to_csv(spath_metals[m] + 'data/fit/' + datafile)

        print('Completed rendering for ' + m.title()) 


    print()
    print('Completed fitting for snapshot ' + str(snap_index) + '\n')

Processing snapshot 68 ... 

Processing C ...


/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in double_scalars
/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until


[ 0.40636044 -1.125       0.22139853 12.73399747 -0.975     ]
Processing N ...
[ 0.39263853 -1.275       0.21063448  9.31107894 -1.175     ]
Processing O ...
[ 0.68007679 -1.925       0.27317359  2.75154311 -1.525     ]
Processing Ne ...
[ 0.57895111 -1.875       0.24136551  2.85484794 -1.425     ]
Processing Mg ...
[ 0.51155145 -2.125       0.23377538  2.25560575 -1.675     ]
Processing Si ...
[ 0.44777396 -2.125       0.19296428  0.90663821 -2.075     ]
Processing S ...
[ 0.75269188 -2.025       0.30846929  2.2415452  -1.725     ]
Processing Ca ...
[nan nan nan nan nan]
Processing Fe ...
[nan nan nan nan nan]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 68

Processing snapshot 69 ... 

Processing C ...
[ 0.7363946  -1.575       0.43002426  5.47191313 -0.925     

[ 0.53335893 -1.425       0.23620987  9.2377021  -1.225     ]
Processing N ...
[ 0.60642106 -1.875       0.27468744  7.34712052 -1.575     ]
Processing O ...
[ 0.52313358 -1.075       0.21839498 11.06077773 -0.975     ]
Processing Ne ...
[ 0.49523902 -1.025       0.20975297 10.41301934 -0.925     ]
Processing Mg ...
[ 0.47209218 -1.275       0.20470506 11.9623871  -1.125     ]
Processing Si ...
[ 0.44701123 -1.275       0.19287242 12.17796014 -1.125     ]
Processing S ...
[ 0.45678558 -1.325       0.19362167 12.1914986  -1.175     ]
Processing Ca ...
[ 0.51493433 -1.575       0.20818571 11.78615034 -1.475     ]
Processing Fe ...
[ 0.3975794  -1.625       0.16850502 12.33615511 -1.525     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 79

Processing snapshot 80 ...

[ 0.65968934 -1.525       0.26840606  5.07591765 -1.475     ]
Processing N ...
[ 0.42120051 -2.075       0.18694758  3.05807005 -1.875     ]
Processing O ...
[ 0.39792889 -1.475       0.16943428  3.46880025 -1.425     ]
Processing Ne ...
[ 0.43311303 -1.425       0.16284021  3.63500258 -1.375     ]
Processing Mg ...
[ 0.31329113 -1.725       0.12418996  3.62114358 -1.625     ]
Processing Si ...
[ 0.39489395 -1.625       0.15333709  7.17435372 -1.375     ]
Processing S ...
[ 0.38434409 -1.675       0.15304951  5.24103478 -1.525     ]
Processing Ca ...
[ 0.36574391 -1.975       0.14754519  5.52841572 -1.775     ]
Processing Fe ...
[ 0.33155543 -1.975       0.14756373  5.3420486  -1.725     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 92

Processing snapshot 93 ...

...

[ 0.42554935 -0.025       0.15758149  4.15471775  0.125     ]
Processing O ...
[0.35173962 0.225      0.13272283 7.96467861 0.525     ]
Processing Ne ...
[0.33502186 0.275      0.12762744        nan        nan]
Processing Mg ...
[ 0.42789277  0.075       0.15661003 19.58131739  0.575     ]
Processing Si ...
[0.37062938 0.075      0.13957194 8.93606684 0.325     ]
Processing S ...
[0.37284859 0.025      0.1401295  8.76340102 0.275     ]
Processing Ca ...
[ 0.32590675 -0.275       0.1228991   6.2325991  -0.175     ]
Processing Fe ...
[ 0.34394593 -0.225       0.12896762  5.74482856 -0.125     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 561

Processing snapshot 562 ... 

Processing C ...
[0.38887926 0.075      0.14387027 5.20990462 0.325     ]
Processing N ...
[ 0

[ 0.39627114 -0.225       0.14340018  5.95402671 -0.175     ]
Processing Fe ...
[ 0.30763615 -0.225       0.11397999  5.44933112 -0.125     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 569

Processing snapshot 570 ... 

Processing C ...
[0.24969416 0.025      0.09441989 4.43806536 0.125     ]
Processing N ...
[ 0.26470915 -0.075       0.10104073  3.56521816  0.025     ]
Processing O ...
[0.21071805 0.175      0.08203187 5.45788833 0.325     ]
Processing Ne ...
[0.40468323 0.325      0.14768289 0.48361173 0.875     ]
Processing Mg ...
[0.38612494 0.075      0.14202892        nan        nan]
Processing Si ...
[0.33709026 0.075      0.12425725 8.71347383 0.325     ]
Processing S ...
[0.33856046 0.025      0.12465537 7.21002525 0.225     ]
Processing Ca ...
[ 0.3892

[ 0.28466215 -0.025       0.11038645  3.42759222 -0.025     ]
Processing O ...
[0.24879912 0.225      0.09660289 6.02328604 0.425     ]
Processing Ne ...
[0.24344032 0.275      0.09429206 7.6213327  0.475     ]
Processing Mg ...
[0.22564562 0.025      0.08858828 5.98563769 0.125     ]
Processing Si ...
[0.18167296 0.025      0.0724498  5.35335371 0.125     ]
Processing S ...
[ 0.18230803 -0.025       0.07267961  4.91718806  0.025     ]
Processing Ca ...
[ 0.22372358 -0.275       0.08773146  5.63218686 -0.175     ]
Processing Fe ...
[ 0.23911356 -0.225       0.09239486  4.67847839 -0.175     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 578

Processing snapshot 579 ... 

Processing C ...
[0.27494193 0.075      0.10419138 4.62352372 0.225     ]
Processing N ...
[ 0

[ 0.14137722 -0.025       0.05894496  4.89882505  0.025     ]
Processing Ca ...
[ 0.18893782 -0.275       0.07567728  5.81757705 -0.175     ]
Processing Fe ...
[ 0.20438745 -0.225       0.08047601  5.34234843 -0.125     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 586

Processing snapshot 587 ... 

Processing C ...
[0.24565409 0.075      0.09365483 4.73940564 0.225     ]
Processing N ...
[ 0.23301421 -0.025       0.09304809  3.90900347  0.125     ]
Processing O ...
[0.21055973 0.225      0.08277204 5.95814149 0.425     ]
Processing Ne ...
[0.20842625 0.275      0.0813221  7.38443427 0.475     ]
Processing Mg ...
[0.18795873 0.025      0.07514716 6.05498253 0.125     ]
Processing Si ...
[0.1384424  0.025      0.05762142 4.90398932 0.075     ]
Processing S ...
[ 0

[0.24682077 0.075      0.0942748  4.60823173 0.225     ]
Processing N ...
[ 0.2389661  -0.025       0.09489686  3.54279943  0.025     ]
Processing O ...
[0.21227693 0.225      0.0836368  6.22856425 0.425     ]
Processing Ne ...
[0.20847777 0.275      0.08178666 7.71886443 0.475     ]
Processing Mg ...
[0.18998508 0.025      0.07596535 5.75417544 0.125     ]
Processing Si ...
[0.14264249 0.025      0.05924178 4.81215684 0.075     ]
Processing S ...
[ 0.14322384 -0.025       0.05945751  4.80292461  0.025     ]
Processing Ca ...
[ 0.1865832  -0.275       0.07483127  5.20548033 -0.225     ]
Processing Fe ...
[ 0.20009757 -0.225       0.07926377  4.86928976 -0.175     ]
Completed rendering for C
Completed rendering for N
Completed rendering for O
Completed rendering for Ne
Completed rendering for Mg
Completed rendering for Si
Completed rendering for S
Completed rendering for Ca
Completed rendering for Fe

Completed fitting for snapshot 595

Processing snapshot 596 ... 

Processing C ...
[0.

In [19]:
for m in metals:
    param_df_init = pd.read_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m))
    param_dict_append = {'A': As[m], 'mu': mus[m], 'sigma': sigmas[m], 
                  'alpha': alphas[m], 'z_T': z_Ts[m]}
    param_df_append = pd.DataFrame(param_dict_append)
    param_df = pd.concat([param_df_init, param_df_append], axis = 1)
    param_df.to_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m), index = False)